## Functions for File Access/Rescaling of Data

In [426]:
def importLabels(filenameString):
    import csv
    import numpy as np
    #Import manual scores
    with open(filenameString, newline='') as csvfile:
        manualScores = list(csv.reader(csvfile))
    #Initialize
    labels = []
    l = len(manualScores) - 2

    #Replaces all blank scores with zeros
    count = 0
    for i in manualScores:
        count = count + 1
        if count > 2:
            if i[1] == '':
                i[1] = 0

    #Create separate list of scores
    for j in range(l):
        labels.append(manualScores[j+2][1])
    return labels

In [427]:
def importPathnames(directoryString):
    import os
    pathnames = []
    filenames = []

    directory = os.fsencode(directoryString)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        #Bubblesort to get pathnames in order
        filenames.append(filename)
        n = len(filenames) 
        # Traverse through all array elements
        for i in range(n): 
        # Last i elements are already in place 
            for j in range(0, n-i-1): 
                sub1 = list(filenames[j])
                sub2 = list(filenames[j+1])
                # traverse the array from 0 to n-i-1 
                # Swap if the element found is greater 
                # than the next element 
                if sub1[24:27] > sub2[24:27] : 
                    filenames[j], filenames[j+1] = filenames[j+1], filenames[j]

    for i in filenames:    
        pathname = directoryString + str(i)
        pathnames.append(pathname) 
    return pathnames

In [428]:
def importFeatures(pathnames,labels):
    #Imports features from csv files
    features = []
    #Get list of file names
    for j in pathnames:
        with open(str(j), newline='') as csvfile:
                kinematics = list(csv.reader(csvfile))
                kinematics = np.asarray(kinematics)
                features.append(kinematics[3:,1:])
    print(np.shape(features))
    labels = labels[0:len(features)]
    return features, labels

In [443]:
def scaleProb(features):
    #Scale all coordinates by associated probability, save new features
    #matrix as pScaledFeatures
    
    #INSTEAD: Go through data and cut out anything with a probability of <.90?
    #But there will be entire frames where several but not all points have a
    #high probability -- instead, omit frames where the cumulative probability of all points is low
    pScaledFeatures = features
    temp = np.empty([73,1291,32])
    #All videos
    icount = 0
    for i in pScaledFeatures: 
        jcount = 0
        #All frames
        for j in i:
            kcount = 0
            sz = len(j)
            #All features
            sumprob = 0
            for k in range(sz-1):
                if ((k+1)%3) == 0:
                    if float(j[k]) < 0.9:
                        #print(float(j[k]))
                        j[k-1] = 0
                        j[k-2] = 0
                    else:
                        print(float(j[k]))
                    #kcount = kcount+2
            #jcount = jcount + 1
            #print(np.shape(temp))
        #icount = icount + 1
    return temp

#temp = scaleProb(features)
#print(np.shape(features))
#print(np.shape(temp))

In [ ]:
temp3 = np.empty([6,6,6])
n, r, c = np.shape(temp3)
for i in range(n):
    for j in range(r):
        for k in range(c):
            temp3[i,j,k] = j
            

#print(temp3)
def deleteRow(temp3):
    n, r, c = np.shape(temp3)
    print(temp3)
    total= 0
    for i in range(n):
        #print(n)
        #print(r)
        #rint(c)
        for j in range(r):
            for k in range(c):
                total= total + temp3[i,j,k]
            print(total)
            if total >18:
                #print(i)
                #print(j)
                #print(k)
                #temp3 = np.delete(temp3,j,1)
                print(np.shape(temp3))
                #temp3 = deleteRow(temp3) 
            total = 0
    return temp3

temp3 = deleteRow(temp3)

In [460]:
def normalize(pScaledFeatures):
    #Scale all coordinates by max pixel value
    maxPix = 551

    normalizedFeatures = pScaledFeatures

    for a in normalizedFeatures:
        for b in a:
            sz = len(b)
            for c in range(sz-1):
                b[c] = float(b[c])/551
    return normalizedFeatures

## Workspace

In [470]:
filenameString = 'R0186_20170815a_manualScores.csv'
labels = importLabels(filenameString)
directoryString = 'sampleData/R0186_20170815a/R0186_20170815a_direct/'
pathnames = importPathnames(directoryString)
features,labels = importFeatures(pathnames,labels)
#pScaledFeatures = scaleProb(features)
normalizedFeatures = normalize(features)
#print(normalizedFeatures)

(73, 1291, 48)


In [471]:
#Do PCA on kinematics data
from sklearn.decomposition import PCA

redDimFeatures = np.empty([73,32])
x = normalizedFeatures[0].transpose()
x = np.asarray(x, dtype=float)
pca = PCA(n_components=32)
pca.fit(x)
x = pca.transform(x)
ycount = 0
for y in normalizedFeatures:
    y = y.transpose()
    y = np.asarray(y, dtype=float)
    redDimFeatures[ycount] = pca.transform(y)[0] 
    ycount= ycount+1

In [301]:
import matplotlib.pyplot as plt

In [472]:
from sklearn.neural_network import MLPClassifier as MLP

#Train a random forest
clf = MLP(solver='sgd', max_iter=10000)
clf.fit(redDimFeatures, labels)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [474]:
testFilenameString = 'R0189_20170924a_manualScores.csv'
testlabels = importLabels(testFilenameString)
directoryString = 'sampleData/R0189_20170924a/R0189_20170924a_direct/'
pathnames = importPathnames(directoryString)
testfeatures,testlabels = importFeatures(testpathnames,testlabels)
#testpScaledFeatures = scaleProb(testfeatures)
testNormalizedFeatures = normalize(testfeatures)
testRedDim = np.empty([testl,32])
ncount = 0
for n in testNormalizedFeatures:
    n = n.transpose()
    n = np.asarray(n,dtype=float)
    testRedDim[ncount] = pca.transform(n)[0]
    ncount = ncount +1
testOutput = clf.predict(testRedDim)

(48, 1291, 48)


In [475]:
print(testlabels)

points = 0
for i in range(len(testlabels)):
    if testlabels[i] == testOutput [i]:
        points = points + 1
        
print(points)
print(points/len(testlabels))

[0, '0', '1', '4', '1', '4', '4', '4', '4', '4', '7', '2', '4', '4', '4', '4', '4', '7', '4', '4', '4', '4', '4', '2', '4', '7', '2', '7', '7', '4', '4', '4', '7', '7', '3', '7', '7', '4', '7', '7', '4', '4', '4', '4', '4', '4', '4', '4']
8
0.16666666666666666


In [476]:
trainingOutput = clf.predict(redDimFeatures)
print(output)
print(np.shape(output))

['0' '4' '1' '1' '1' '1' '4' '1' '1' '4' '1' '4' '1' '1' '1' '1' '4' '1'
 '1' '1' '1' '4' '1' '1' '4' '4' '1' '1' '1' '1' '1' '1' '4' '1' '4' '1'
 '4' '1' '1' '1' '1' '1' '4' '1' '1' '4' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '4' '1' '1' '1' '1' '1' '1' '4' '1' '1' '1' '1' '1' '1' '1' '4'
 '1']
(73,)


In [477]:
print(labels)
print(np.shape(labels))

[0, '1', '4', '4', '1', '1', '4', '1', '2', '4', '1', '4', '1', '1', '1', '4', '4', '1', '1', '1', '1', '4', '1', '1', '4', '4', 0, '4', '1', '1', '1', 0, '4', '1', '4', '1', '4', '1', '4', '4', 0, '1', '4', '1', '1', '4', '4', '1', '1', '1', '4', '1', 0, '4', '1', '1', '4', '1', '1', '4', 0, '1', '1', '4', '1', '1', 0, '3', '1', '4', '1', '2', '4']
(73,)


In [489]:
points = 0
for i in range(len(labels)):
    if labels[i] == trainingOutput [i]:
        points = points + 1
        
print(points)
print(points/len(labels))

29
0.6041666666666666


In [492]:
filenameString = 'R0189_20170924a_manualScores.csv'
labels = importLabels(filenameString)
vid, fram, feat = np.shape(normalizedFeatures)
directoryString = 'sampleData/R0189_20170924a/R0189_20170924a_direct/'
pathnames = importPathnames(directoryString)
features,labels = importFeatures(pathnames,labels)
#pScaledFeatures = scaleProb(features)
normalizedFeatures = normalize(features)

redDimFeatures = np.empty([vid,32])
x = normalizedFeatures[0].transpose()
x = np.asarray(x, dtype=float)
pca2 = PCA(n_components=32)
pca2.fit(x)
x = pca2.transform(x)
ycount = 0
for y in normalizedFeatures:
    y = y.transpose()
    y = np.asarray(y, dtype=float)
    redDimFeatures[ycount] = pca2.transform(y)[0] 
    ycount= ycount+1
    
clf2 = MLP(solver='sgd', max_iter=10000)
clf2.fit(redDimFeatures, labels)

testFilenameString = 'R0186_20170815a_manualScores.csv'
testlabels = importLabels(testFilenameString)
vid2, fram2, feat2 = np.shape(normalizedFeatures)
directoryString = 'sampleData/R0186_20170815a/R0186_20170815a_direct/'
pathnames = importPathnames(directoryString)
testfeatures,testlabels = importFeatures(testpathnames,testlabels)
#testpScaledFeatures = scaleProb(testfeatures)
testNormalizedFeatures = normalize(testfeatures)
testRedDim = np.empty([vid2,32])
ncount = 0
for n in testNormalizedFeatures:
    n = n.transpose()
    n = np.asarray(n,dtype=float)
    testRedDim[ncount] = pca2.transform(n)[0]
    ncount = ncount +1
testOutput = clf2.predict(testRedDim)

print(testlabels)

points = 0
for i in range(len(testlabels)):
    if int(testlabels[i]) == int(testOutput [i]):
        points = points + 1
        
print(points)
print(points/len(testlabels))

trainingOutput = clf2.predict(redDimFeatures)
print(output)
print(np.shape(output))

print(labels)
print(np.shape(labels))

points = 0
for i in range(len(labels)):
    if int(labels[i]) == int(trainingOutput[i]):
        points = points + 1
        
print(points)
print(points/len(labels))

(48, 1291, 48)
(48, 1291, 48)
[0, '1', '4', '4', '1', '1', '4', '1', '2', '4', '1', '4', '1', '1', '1', '4', '4', '1', '1', '1', '1', '4', '1', '1', '4', '4', 0, '4', '1', '1', '1', 0, '4', '1', '4', '1', '4', '1', '4', '4', 0, '1', '4', '1', '1', '4', '4', '1']
12
0.25
['0' '4' '1' '1' '1' '1' '4' '1' '1' '4' '1' '4' '1' '1' '1' '1' '4' '1'
 '1' '1' '1' '4' '1' '1' '4' '4' '1' '1' '1' '1' '1' '1' '4' '1' '4' '1'
 '4' '1' '1' '1' '1' '1' '4' '1' '1' '4' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '4' '1' '1' '1' '1' '1' '1' '4' '1' '1' '1' '1' '1' '1' '1' '4'
 '1']
(73,)
[0, '0', '1', '4', '1', '4', '4', '4', '4', '4', '7', '2', '4', '4', '4', '4', '4', '7', '4', '4', '4', '4', '4', '2', '4', '7', '2', '7', '7', '4', '4', '4', '7', '7', '3', '7', '7', '4', '7', '7', '4', '4', '4', '4', '4', '4', '4', '4']
(48,)
48
1.0


## Test Left View

In [322]:
filenameString = 'R0186_20170815a_manualScores.csv'
labels = importLabels(filenameString)
directoryString = 'sampleData/R0186_20170815a/R0186_20170815a_left/'
pathnames = importPathnames(directoryString)
features,labels = importFeatures(pathnames,labels)
pScaledFeatures = scaleProb(features)
normalizedFeatures = normalize(pScaledFeatures)

(73, 1291, 48)


In [323]:
from sklearn.neural_network import MLPClassifier as MLP

#Train a random forest
clf1 = MLP(solver='sgd', max_iter=10000)
clf1.fit(redDimFeatures, labels)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [324]:
testFilenameString = 'R0189_20170924a_manualScores.csv'
testlabels = importLabels(testFilenameString)
directoryString = 'sampleData/R0189_20170924a/R0189_20170924a_left/'
pathnames = importPathnames(directoryString)
testfeatures,testlabels = importFeatures(testpathnames,testlabels)

testNormalizedFeatures = normalize(testpScaledFeatures)
testRedDim = np.empty([testl,32])
ncount = 0
for n in testNormalizedFeatures:
    n = n.transpose()
    testRedDim[ncount] = pca.transform(n)[0]
    ncount = ncount +1
testOutput = clf1.predict(testRedDim)

(48, 1291, 48)


In [325]:
print(testlabels)

points = 0
for i in range(len(testlabels)):
    if testlabels[i] == testOutput [i]:
        points = points + 1
        
print(points)
print(points/len(testlabels))

trainingOutput = clf1.predict(redDimFeatures)
print(output)
print(np.shape(output))

points = 0
for i in range(len(labels)):
    if labels[i] == trainingOutput [i]:
        points = points + 1
        
print(points)
print(points/73)

[0, '0', '1', '4', '1', '4', '4', '4', '4', '4', '7', '2', '4', '4', '4', '4', '4', '7', '4', '4', '4', '4', '4', '2', '4', '7', '2', '7', '7', '4', '4', '4', '7', '7', '3', '7', '7', '4', '7', '7', '4', '4', '4', '4', '4', '4', '4', '4']
2
0.041666666666666664
['0' '4' '1' '1' '1' '1' '4' '1' '1' '4' '1' '4' '1' '1' '1' '1' '4' '1'
 '1' '1' '1' '4' '1' '1' '4' '4' '1' '1' '1' '1' '1' '1' '4' '1' '4' '1'
 '4' '1' '1' '1' '1' '1' '4' '1' '1' '4' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '4' '1' '1' '1' '1' '1' '1' '4' '1' '1' '1' '1' '1' '1' '1' '4'
 '1']
(73,)
66
0.9041095890410958
